<a href="https://colab.research.google.com/github/cwagner2325/WiseInbox_ML/blob/main/Wise_Inbox_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio

In [40]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import random
import os
from google.colab import userdata
from pyngrok import conf

In [41]:
app = FastAPI()

In [42]:
origins = ["*"] # Change this to local host probably

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [43]:
class model_input(BaseModel):

    Email : str
    Sender: str
    Subject : str

In [44]:
# Example I found online. This guy used pickle to export a ML model as a file
# And then open the model and store it as a variable again using Pickle

# email_model = pickle.load(open('email_model.sav', 'rb'))

In [45]:
@app.post('/email_prediction')
def email_prediction(input_parameters : model_input):

    print("Email Prediction Reached")
    input_data = input_parameters.json()
    input_dictionary = json.loads(input_data)

    email = input_dictionary['Email']
    sender = input_dictionary['Sender']
    subject = input_dictionary['Subject']


    input_list = [email, sender, subject]

    prediction = random.randint(60, 100)
    print(prediction)

    # prediction = email_model.predict([input_list])


    return prediction

In [47]:
os.environ["TOKEN"] = userdata.get("TOKEN")
ngrok.set_auth_token(os.environ["TOKEN"])

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://27d8-35-245-119-48.ngrok-free.app


INFO:     Started server process [249]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [249]
